In [27]:
# Import required libraries
import sqlite3
import numpy as np
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [28]:
# Read data
conn = sqlite3.connect('restaurant.sqlite')

# Use a query to load the entire table 
restaurant_df = pd.read_sql("SELECT * FROM restaurants", conn)

# Display sample data
restaurant_df.head()

,Name,License_ID,Risk,zip,Inspection_Date,Results,Violations
0,CHIPOTLE MEXICAN GRILL,2670642.0,Risk 1 (High),60613.0,2019,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
1,SUBWAY,2703766.0,Risk 1 (High),60661.0,2019,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
2,SWEET SHOT COOKIES,2688916.0,Risk 2 (Medium),60647.0,2019,Pass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...
3,CHIPOTLE MEXICAN GRILL,2670642.0,Risk 1 (High),60613.0,2019,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."
4,SAME SAME,2689715.0,Risk 1 (High),60618.0,2019,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E..."


In [29]:
# Convert Risk into numerical values
risk_mapping = {
    'Risk 1 (High)': 3,
    'Risk 2 (Medium)': 2,
    'Risk 3 (Low)': 1
}
restaurant_df['Risk_numeric'] = restaurant_df['Risk'].map(risk_mapping)

# Convert Results into dummy variables
results_dummies = pd.get_dummies(restaurant_df['Results'], prefix='Results')
results_dummies = results_dummies * 1  # Convert True/False to 1/0 if needed
restaurant_df = pd.concat([restaurant_df, results_dummies], axis=1)

# Drop boolean columns
for col in restaurant_df.columns:
    if restaurant_df[col].dtype == bool:
        restaurant_df.drop(col, axis=1, inplace=True)

restaurant_df.head()

,Name,License_ID,Risk,zip,Inspection_Date,Results,Violations,Risk_numeric,Results_Fail,Results_No Entry,Results_Not Ready,Results_Pass,Results_Pass w/ Conditions
0,CHIPOTLE MEXICAN GRILL,2670642.0,Risk 1 (High),60613.0,2019,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",3,0,0,0,0,1
1,SUBWAY,2703766.0,Risk 1 (High),60661.0,2019,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",3,0,0,0,0,1
2,SWEET SHOT COOKIES,2688916.0,Risk 2 (Medium),60647.0,2019,Pass,49. NON-FOOD/FOOD CONTACT SURFACES CLEAN - Com...,2,0,0,0,1,0
3,CHIPOTLE MEXICAN GRILL,2670642.0,Risk 1 (High),60613.0,2019,Fail,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",3,1,0,0,0,0
4,SAME SAME,2689715.0,Risk 1 (High),60618.0,2019,Pass w/ Conditions,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",3,0,0,0,0,1


In [30]:
# Drop non-numeric columns (like restaurant names, addresses, etc.)
# This step assumes that such columns are of 'object' dtype. Modify if needed.
X_df = restaurant_df.select_dtypes(exclude=['object']).drop("Results_Fail", axis=1)

# For columns with a small number of unique values, use label encoding
# You might want to adjust the threshold (e.g., 10) based on your data
for col in X_df.columns:
    if len(X_df[col].unique()) <= 10:
        le = LabelEncoder()
        X_df[col] = le.fit_transform(X_df[col])

bestfeatures = SelectKBest(score_func=chi2, k=5)
fit = bestfeatures.fit(X_df,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_df.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Features','Score']

print(featureScores.nlargest(5,'Score'))

                     Features          Score
0                  License_ID  313536.513891
2                Risk_numeric    1376.140043
5                Results_Pass       3.031039
4           Results_Not Ready       3.003553
6  Results_Pass w/ Conditions       1.028840


In [33]:
# Using the top features for prediction
X = restaurant_df[['License_ID', 'Risk_numeric']]
y = restaurant_df["Results_Fail"]

# Splitting data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create & fit model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate R^2 score
r2 = r2_score(y_test, y_pred)
print(f"R-squared value: {r2}")

R-squared value: 0.084380415463298
